In [1810]:
# % matplotlib inline

import numpy as np
from sklearn.svm import SVC
from matplotlib import cm
import matplotlib.pyplot as plt

plt.style.use(plt.style.available[0])  #可以通过下标进行选择适合自己的画图样式
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification

In [1812]:
# 两个信息特性，每个类一个集群
X, y = make_classification(n_samples=50,  #(default=100)样本数
                           n_features=3,  #(default=20)特征序总数。这些包括随机绘制的n_informative信息特征，n_redundant冗余特征，n_repeated重复特征和n_features-n_informative-n_redundant-n_repeated无用特征。
                           n_redundant=0,  #(default=2)冗余特征数量
                           n_repeated=0,  #(default=0)从信息性和冗余性特征中随机抽取的重复性特征的数量。
                           n_informative=1,  #有效特征个数
                           n_clusters_per_class=1,  #(default=2)每个类的簇数
                           n_classes=2,  #(default=2)分类问题的类（或标签）数
                           flip_y=0.8,  #(default=0.01)类别随机分配的样本比例
                           random_state=2,  #确定用于生成数据集的随机数生成。 为多个函数调用传递可重复输出的int值。
                           )


In [1813]:
print('X= ', X[:3])
print('y= ', y[:5])
print(X.shape, X.ndim)
print(y.shape, y.ndim)

X=  [[ 0.41819493  0.69619798  0.69741627]
 [-0.42896232 -0.63467931  0.50839624]
 [-1.61274204 -1.09873895  1.58448706]]
y=  [1 1 1 1 0]
(50, 3) 2
(50,) 1


In [1814]:
#将数据划分为训练集和验证集
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [1817]:
# 3. 特征工程（标准化）
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()

# 测试集 计算平均值和标准偏差 缩放特征集
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.transform(X_test)
# X = standardScaler.fit_transform(X)


In [1818]:
# Fit regression model
classifiers = [
    SVC(kernel="rbf", C=1,  #浮点数，默认= 1.0 正则化参数。正则化的强度与C成反比。必须严格为正。此惩罚系数是l2惩罚系数的平方
        gamma=.5,  #浮点数或者{‘scale’, ‘auto’} , 默认=’scale’ 核系数包含‘rbf’, ‘poly’ 和‘sigmoid’
        # 如果gamma='scale'(默认)，则它使用1 / (n_features * X.var())作为gamma的值，如果是auto，则使用1 / n_features。在0.22版本有改动：默认的gamma从“auto”改为“scale”。
        probability=True,  #是否启用概率估计。必须在调用fit之前启用此参数，因为该方法内部使用5折交叉验证，因此会减慢该方法的速度
        ),
    SVC(kernel="linear", C=1, gamma=1, probability=True),
    SVC(kernel="poly", C=1, gamma="auto", degree=3, coef0=1),
]

names = [
    "RBF SVM",
    "Linear SVM",
    "poly"
]

In [1819]:
for name, clf in zip(names, classifiers):
    performance = clf.fit(X_train, y_train.ravel())
    # 获取预测值
    y_test_pred = performance.predict(X_test)
    # 显示估计器
    print(f'%s, svcs=%s' % (name, clf))
    # 获取这个估计器的参数
    print(f'估计器的参数: %s' % (clf.get_params()))
    # https://blog.csdn.net/gracejpw/article/details/101546293
    # 返回预测的决定系数R^2
    # R^2越接近于1，模型的拟合优度越高。
    print(f'训练集R2评分: %.2f ' % (clf.score(X_train, y_train)))
    print(f'测试集R2评分: %.2f ' % (clf.score(X_test, y_test)))


RBF SVM, svcs=SVC(C=1, gamma=0.5, probability=True)
估计器的参数: {'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.5, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
训练集R2评分: 0.78 
测试集R2评分: 0.92 
Linear SVM, svcs=SVC(C=1, gamma=1, kernel='linear', probability=True)
估计器的参数: {'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'linear', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
训练集R2评分: 0.65 
测试集R2评分: 0.77 
poly, svcs=SVC(C=1, coef0=1, gamma='auto', kernel='poly')
估计器的参数: {'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 1, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': -1, 'probabi

In [1878]:
#设定numpy显示浮点数精度的小数位数
np.set_printoptions(precision=3)
# 核函数返回参数介绍
# clf所使用的分类器
clf = classifiers[1]
print('kernel 核函数名称', clf.kernel)  #核函数名称
print('support_ 形如%s的数组,支持向量的下标,显示前10个 \n=%s'
      % (clf.support_.shape, clf.support_[:10]))  #形如(n_SV,)的数组,支持向量的指标。
print('support_vectors_ 形如%s的数组,支持向量, 显示前3行 \n=%s'
      %(clf.support_vectors_.shape, clf.support_vectors_[:3]))  #形如(n_SV, n_features)的数组, 支持向量
print('n_support_ 形如%s的数组,每个类别的支持向量数量, \n=%s'
      %(clf.n_support_.shape, clf.n_support_))  #形如(n_class)的数组，每个类别的支持向量数量。
print('dual_coef_ 形如%s的数组,决策函数中支持向量的对偶系数, \n=%s'  #形如(n_class-1, n_SV)的数组, 决策函数中支持向量的对偶系数
      %(clf.dual_coef_.shape, clf.dual_coef_[:3,:5]))  #显示前三个类型的前3个系数
print(#clf.coef_ 形如(n_class * (n_class-1) / 2, n_features)的数组,分配给特征的权重（原始问题的系数），仅在线性内核的情况下可用。
    ('coef_ 形如%s的数组,分配给特征的权重, \n=%s，仅在线性内核的情况下可用'%(clf.coef_.shape, clf.coef_)) if clf.kernel == 'linear' else 0)
print('intercept_ 形如%s的数组,添加到决策函数的常量（也称为偏差或截距）, \n=%s'%(clf.intercept_.shape,clf.intercept_))  #形如(n_class * (n_class-1) / 2,)的数组,决策函数中的常量。
print('fit_status_ 如果拟合无误，则为0；如果算法未收敛，则为1,fit_status_=',clf.fit_status_)  #整数型 如果拟合无误，则为0；如果算法未收敛，则为1
print('classes_ 形如%s的数组 不重复的类别标签, \n=%s'  #二分类情况下就只有2个标签
      %(clf.classes_.shape, clf.classes_))  #形如(n_classes,)的数组 不重复类别标签
print('probA_ 形如(%s * (%s-1) / 2,)=%s的数组, \n=%s, 如果 probability=False，则为空数组。'
      %(clf.classes_.size,clf.classes_.size,clf.probA_.shape, clf.probA_)
      )  #probA_ 形如(n_class * (n_class-1) / 2,)的数组
'''如果 probability=True，则它对应于在普拉特缩放中学习的参数，以根据决策值产生概率估计。

普拉特定标使用逻辑函数1 /（1 + exp（decision_value * probA_ + probB_）），
其中从数据集[2]了解probA_和probB_。有关多类案件和培训程序的更多信息，请参见[1]的第8节。'''
print('probB_ 形如(%s * (%s-1) / 2,)=%s的数组, \n=%s, 如果 probability=False，则为空数组。'
      %(clf.classes_.size,clf.classes_.size,clf.probB_.shape, clf.probB_))

print('class_weight_ 形如(%s,)的数组每个类的参数C的乘数。根据 class_weight 参数进行计算。\n=%s'
      %(clf.classes_.size, clf.class_weight_)
      )  #形如(n_class,)的数组每个类的参数C的乘数。根据class_weight参数进行计算。

print('shape_fit_ 形如(n_dimensions_of_X,)的整数型元祖 训练向量X的数组维度', clf.shape_fit_)

kernel 核函数名称 linear
support_ 形如(28,)的数组,支持向量的下标,显示前10个 
=[ 5  8  9 11 12 14 16 19 20 21]
support_vectors_ 形如(28, 3)的数组,支持向量, 显示前3行 
=[[-0.595 -0.527  0.736]
 [-0.733  1.032  0.289]
 [-0.634 -0.363  0.254]]
n_support_ 形如(2,)的数组,每个类别的支持向量数量, 
=[14 14]
dual_coef_ 形如(1, 28)的数组,决策函数中支持向量的对偶系数, 
=[[-1. -1. -1. -1. -1.]]
coef_ 形如(1, 3)的数组,分配给特征的权重, 
=[[1.078 0.06  0.237]]，仅在线性内核的情况下可用
intercept_ 形如(1,)的数组,添加到决策函数的常量（也称为偏差或截距）, 
=[0.068]
fit_status_ 如果拟合无误，则为0；如果算法未收敛，则为1,fit_status_= 0
classes_ 形如(2,)的数组 不重复的类别标签, 
=[0 1]
probA_ 形如(2 * (2-1) / 2,)=(1,)的数组, 
=[-0.394], 如果 probability=False，则为空数组。
probB_ 形如(2 * (2-1) / 2,)=(1,)的数组, 
=[0.128], 如果 probability=False，则为空数组。
class_weight_ 形如(2,)的数组每个类的参数C的乘数。根据 class_weight 参数进行计算。
=[1. 1.]
shape_fit_ 形如(n_dimensions_of_X,)的整数型元祖 训练向量X的数组维度 (37, 3)


In [1924]:
print('classes_ 类别标签 =',clf.classes_)
print('predict_proba 返回测试样本的各类标签 概率. 每行概率合计为 1 \n', clf.predict_proba(X_test)[:10])
print('predict 返回测试样本的所属类别标签\npredict', clf.predict(X_test)[:])
print('y_test ',y_test[:])
id_test = clf.predict(X_test) != y_test  #测试样本预测类别与测试目标类别不同的下标
print('测试样本预测类别与测试目标类别不同的由predict_proba计算的概率值\n',clf.predict_proba(X_test)[id_test])
print('predict 返回训练样本的所属类别标签\npredict', clf.predict(X_train)[:])
print('y_train',y_train[:])

classes_ 类别标签 = [0 1]
predict_proba 返回测试样本的各类标签 概率. 每行概率合计为 1 
 [[0.47  0.53 ]
 [0.333 0.667]
 [0.582 0.418]
 [0.433 0.567]
 [0.578 0.422]
 [0.418 0.582]
 [0.392 0.608]
 [0.352 0.648]
 [0.393 0.607]
 [0.295 0.705]]
predict 返回测试样本的所属类别标签
predict [0 1 0 1 0 1 1 1 1 1 1 0 1]
y_test  [1 1 0 0 1 1 1 1 1 1 1 0 1]
测试样本预测类别与测试目标类别不同的由predict_proba计算的概率值
 [[0.47  0.53 ]
 [0.433 0.567]
 [0.578 0.422]]
predict 返回训练样本的所属类别标签
predict [1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1]
y_train [1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0]
训练样本预测类别与训练目标类别不同的由predict_proba计算的概率值
 [[0.471 0.529]
 [0.531 0.469]
 [0.509 0.491]
 [0.369 0.631]
 [0.543 0.457]
 [0.354 0.646]
 [0.535 0.465]
 [0.5   0.5  ]
 [0.546 0.454]
 [0.494 0.506]
 [0.564 0.436]
 [0.486 0.514]
 [0.285 0.715]]


In [1939]:
id_train_no = clf.predict(X_train)[:] != y_train  #训练样本预测类别与训练目标类别不同的下标
id_train_ok = clf.predict(X_train)[:] == y_train  #训练样本预测类别与训练目标类别不同的下标
print('训练样本预测类别与训练目标类别不同的由predict_proba计算的概率值\n', clf.predict_proba(X_train)[id_train_no])

proba_no = clf.predict_proba(X_train)[id_train_no]  #训练样本预测类别与训练目标类别不同的由predict_proba计算的概率值
proba_ok = clf.predict_proba(X_train)[id_train_ok]
print(proba_no.shape)

# 统计预测类别不同的数据在5个等分区间内的个数,由于统计的是数组中所有值,所以统计个数是行*列的值
proba_his_no = np.histogram(proba_no,bins=5)
print('预测类别不同的区间个数统计%s\n概率统计区间分布%s'% proba_his_no)
proba_his_ok = np.histogram(proba_ok,bins=5)
print('预测类别相同的区间个数统计%s\n概率统计区间分布%s'% (proba_his_ok))


训练样本预测类别与训练目标类别不同的由predict_proba计算的概率值
 [[0.471 0.529]
 [0.531 0.469]
 [0.509 0.491]
 [0.369 0.631]
 [0.543 0.457]
 [0.354 0.646]
 [0.535 0.465]
 [0.5   0.5  ]
 [0.546 0.454]
 [0.494 0.506]
 [0.564 0.436]
 [0.486 0.514]
 [0.285 0.715]]
(13, 2)
预测类别不同的区间个数统计[ 3  2 16  2  3]
概率统计区间分布[0.285 0.371 0.457 0.543 0.629 0.715]
预测类别相同的区间个数统计[ 5 13 12 13  5]
概率统计区间分布[0.245 0.347 0.449 0.551 0.653 0.755]


In [1821]:
print(X.shape, X.ndim)
print('standardScaler.mean_', standardScaler.mean_)  #均值
print('standardScaler.var_', standardScaler.var_)  #方差
print('x_train.var', np.var(X_train))  #方差
print('x_train.std', np.std(X_train))  #标准差
print('X_train=', X_train[:3])
print('X_test=', X_test[:3])
print('y_train=', y_train[:3])
print('y_test=', y_test[:3])


(50, 3) 2
standardScaler.mean_ [-0.1041486  -0.03981066  0.09162495]
standardScaler.var_ [1.20553777 1.35602197 1.0400875 ]
x_train.var 1.0000000000000002
x_train.std 1.0
X_train= [[ 0.90753027 -0.2449489  -0.6391739 ]
 [-0.1882497  -1.2388795   0.7400003 ]
 [-0.77783026 -1.45828494  0.93072705]]
X_test= [[ 0.09734242 -0.58347168 -0.66190289]
 [ 1.24237466 -0.84295228  0.33413916]
 [-0.81125864 -0.312655   -1.44741702]]
y_train= [1 1 1]
y_test= [1 1 0]
